In [ ]:
# This program obtains timestamps from our videos marking when asterisks appear on our ATM
# @author: Denisolt Shakhbulatov, Kendall Molas, Tristan Gurtler
from PIL import Image
import sys
import pandas as pd
import os
import csv

In [ ]:
###
# Input the file names
# @input startTime - Start video from a certain time-frame, user-determined
# @input videoFileName - Input video file name
# @input imageFileSaveArea - Folder where extracted images will be saved
# @input frameFileName - CSV file that will contain timestamp data
# @input contrastVal - contrast at which images will be changed to, determined empirically
# @input brightnessVal - increase brightness of certain videos, determined empirically
# @input cropImage - [x,y,z,w] | [x,y] is the image size, and [z,w] is the coordinate on the image where it will start creating
# the size image
###

startTime = ''
videoFileName = ''
videoFileExtension = '.MP4'
videoToBeProcessed = videoFileName + videoFileExtension
imageFileSaveArea = 'images' + videoFileName
frameFileName = 'frames' + videoFileName + '.csv'
contrastVal = '35'
brightnessVal = ''
cropImage = ''

In [ ]:
# Creates directory for frames to be stored, gets frames of videos, and stores timestamps in a csv file
try:
    os.system("mkdir ../" + imageFileSaveArea)
    #Standard for processing
    os.system("ffmpeg -ss " + startTime + " -i ../" + videoToBeProcessed + " -an -vf crop=" + cropImage + ",eq=contrast=" + contrastVal + " ../" + imageFileSaveArea + "/%06d.png")
   
    # Use this if video is too dark when processing
    #os.system("ffmpeg -ss " + startTime + " -i ../" + videoToBeProcessed + " -an -vf crop=" + cropImage + ",eq=contrast=" + contrastVal + ":brightness=" + brightnessVal + " ../" + imageFileSaveArea + "/%06d.png")
   
    # May want to use this for the purpose of finding area of where to get coordinates for cropping the image
    #os.system("ffmpeg -ss " + startTime + " -i ../" + videoToBeProcessed + " -an -vf crop=" + cropImage + " ../" + imageFileSaveArea + "/%06d.png")
    
    # This always needs to be on
    os.system("ffprobe -f lavfi -i movie=../" + videoToBeProcessed + " -show_frames -show_entries frame=pkt_pts_time -of csv=p=0 > ../" + frameFileName)
    print 'Successful'
except:
    print 'ERROR'

In [ ]:
##
# This function checks if the image we are looking at is actually a PIN entry screen at all
#
# @input image - the image we want to check for being a PIN Entry screen
# @returns whether or not we are in a PIN Entry screen

def determine_background(image, coord_topleft, coord_bottomright):
    total_pixels = 0
    num_white_pixels = 0

    # range through our box and check for white pixels
    for x in range(coord_topleft[0], coord_bottomright[0] + 1):
        for y in range(coord_topleft[1], coord_bottomright[1] + 1):
            curr = (x,y)
            r, g, b = image.getpixel(curr)
            if ((r,g,b) == (255,255,255)):
                num_white_pixels += 1
            total_pixels += 1
    if ((float(num_white_pixels) / total_pixels) == 1):
        return False
    else:
        return True

In [ ]:
##
# This function checks at specified locations to determine if an asterisk is present there
#
# @input image - the image we want to check if an asterisk has appeared inside of
# @input coord_topleft - the top left corner of the box we are checking
# @input coord_bottomright - the bottom right corner of the box we are checking
# @input threshold - what percentage of pixels should be black to determine that an asterisk has appeared
# @returns whether or not an asterisk has appeared at the specified location

def determine_hash(image, coord_topleft, coord_bottomright, threshold=0.00857):
    total_pixels = 0
    num_black_pixels = 0

    # range through our box and check for black pixels
    for x in range(coord_topleft[0], coord_bottomright[0] + 1):
        for y in range(coord_topleft[1], coord_bottomright[1] + 1):
            curr = (x,y)
            r, g, b = image.getpixel(curr)
            if is_color_black(r, g, b):
                num_black_pixels += 1
            total_pixels += 1

    return (float(num_black_pixels) / total_pixels) >= threshold

In [ ]:
# This function checks if a given color (by rgb values) is black or gray
def is_color_black(r, g, b):
    flag_color_is_grayscale = r == g and g == b
    flag_color_is_dark = r <= 211 and g <= 211 and b <= 211
    
    return flag_color_is_grayscale and flag_color_is_dark

In [ ]:
# Initialize data frame where the values of the asterisks will be stored
df = pd.DataFrame()
# Begin of time_stamp
time_stamp = 0
# Match the image numbers
frame = 1
indir = '../' + imageFileSaveArea

##
# Locations for scanning asterisk in the area before the shift occurs
##
first_asterisk_area_before_top_left  = (,)
first_asterisk_area_before_bottom_right = (,)

second_asterisk_area_before_top_left = (,)
second_asterisk_area_before_bottom_right = (,)

third_asterisk_area_before_top_left = (,)
third_asterisk_area_before_bottom_right = (,)

fourth_asterisk_area_before_top_left = (,)
fourth_asterisk_area_before_bottom_right = (,)

##
# Locations for scanning asterisk in the other area, after the shift
##
first_asterisk_area_after_top_left  = (,)
first_asterisk_area_after_bottom_right = (,)

second_asterisk_area_after_top_left = (,)
second_asterisk_area_after_bottom_right = (,)

third_asterisk_area_after_top_left = (,)
third_asterisk_area_after_bottom_right = (,)

fourth_asterisk_area_after_top_left = (,)
fourth_asterisk_area_after_bottom_right = (,)

background_topleft = (,)
background_bottomright = (,)

##
# Read through the files in the sorted directory and determine whether hash has appeared or not
##
for root, dirs, filenames in os.walk(indir):
    for f in sorted(os.listdir(indir)):
        log = open(os.path.join(root, f), 'r')
        im = Image.open(log)
        pix = im.load()
        rgb_im = im.convert('RGB')    
        first_asterisk = determine_hash(rgb_im, first_asterisk_area_before_top_left,first_asterisk_area_before_bottom_right) or determine_hash(rgb_im,first_asterisk_area_after_top_left, first_asterisk_area_after_bottom_right)
        second_asterisk = determine_hash(rgb_im, second_asterisk_area_before_top_left,second_asterisk_area_before_bottom_right) or determine_hash(rgb_im,second_asterisk_area_after_top_left, second_asterisk_area_after_bottom_right)
        third_asterisk = determine_hash(rgb_im, third_asterisk_area_before_top_left,third_asterisk_area_before_bottom_right) or determine_hash(rgb_im,third_asterisk_area_after_top_left, third_asterisk_area_after_bottom_right)
        fourth_asterisk = determine_hash(rgb_im, fourth_asterisk_area_before_top_left,fourth_asterisk_area_before_bottom_right) or determine_hash(rgb_im,fourth_asterisk_area_after_top_left, fourth_asterisk_area_after_bottom_right)
        
        ## 
        # @input background - coordinates are empirically determined
        ##
        
        background = determine_background(rgb_im, background_topleft, background_bottomright)
        
        ##
        # Cause asterisks to return empty if False
        ##
        if (first_asterisk == False):
            first_asterisk = ''
        if (second_asterisk == False):
            second_asterisk = ''
        if (third_asterisk == False):
            third_asterisk = ''
        if (fourth_asterisk == False):
            fourth_asterisk = ''
        
        ##
        # Checks if background is black or white
        # If False - Asterisks will pop up with True/False values
        # Else - Columns will be filled with NaNs
        
        if(background == False):
            df = df.append(pd.DataFrame({'Background': background, 'First': first_asterisk, 'Second': second_asterisk, 'Third': third_asterisk, 'Fourth': fourth_asterisk, }, index=[frame]), ignore_index=False)
        else:
            df = df.append(pd.DataFrame({'Background': background, 'First': 'NaN', 'Second': 'NaN', 'Third': 'NaN', 'Fourth': 'NaN', }, index=[frame]), ignore_index=False)
        frame = frame + 1
        if (frame % 1000 == 0):
            print frame

In [ ]:
# Reads from frames file and combines to have timestamp and dataframe together
Time = pd.read_csv("../" + frameFileName, 
                  names = ["Time_stamp"])
result = pd.concat([df, Time], axis=1, join='inner')

outputFileForUser = 'results' + videoFileName + '.csv'

result = result.reindex(columns=['Time_stamp','Background', 'First', 'Second', 'Third', 'Fourth']).to_csv('../' + outputFileForUser, index=True)
result

In [ ]:
outputFileForUser = 'results' + videoFileName + '.csv'
newReader = pd.read_csv('../' + outputFileForUser, index_col=0)

newReader.loc[:, 'and'] = newReader.First & newReader.Second & newReader.Third & newReader.Fourth
newReader.loc[:, 'next_and'] = newReader['and'].shift(-1)
newReader.loc[:, 'clear'] = newReader['and'] & newReader.next_and


In [ ]:
# Boolean to find the first keypress and prevent from printing out every single line of the csv file to output

firstKP = False
secondKP = False
thirdKP = False
fourthKP = False
clr = False
entr = False

try:
    with open('../relevantKeyPresses' + videoFileName + '.txt', 'w') as textOutput:
        # Iterate through every row in the csv file and check specific columns for True values
        for index, row in newReader.iterrows():
            x = index

            ## First Asterisk Occurance
            # Checks first column for True
            if (newReader['First'].iloc[index] == True and pd.isnull(newReader['Second'].iloc[index]) == True
               and pd.isnull(newReader['Third'].iloc[index]) == True and pd.isnull(newReader['Fourth'].iloc[index]) == True):

               # Check to see if a first keypress was already found
                if (firstKP == False):
                    if (clr == True):
                        clr = False
                    print newReader.Time_stamp.iloc[[index]]
                    print'x'
                    #textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    firstKP = True

                # If the user pressed enter before the four PIN sequence is completed
                # Display time at which user presses ENTER 
                if (newReader['Background'].iloc[x+1] == True and firstKP == True):
                    print newReader.Time_stamp.iloc[[index]]
                    print'xx'
                    #textOutput.write('%s\n' % newReader.Time_stamp.iloc[[x+1]])
                    textOutput.write('^\n')
                    firstKP = False

            # If a first keypress was already found, and becomes missing due to a CLEAR, reset boolean
            elif (firstKP == True and pd.isnull(newReader['First'].iloc[index]) == True):
                clr = True
                firstKP = False

            ## Second Asterisk Occurance
            # Check second column for True
            if (newReader['Second'].iloc[index] == True and pd.isnull(newReader['Third'].iloc[index]) == True
                 and pd.isnull(newReader['Fourth'].iloc[index]) == True):

                # Check to see if the second keypress was already found
                if (secondKP == False):
                    if (clr == True):
                        clr = False

                    print newReader.Time_stamp.iloc[[index]]
                    print'y'
                    #textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    secondKP = True

                # If the user pressed enter before the four PIN sequence is completed
                if (newReader['Background'].iloc[x+1] == True and secondKP == True and firstKP == True):
                    print newReader.Time_stamp.iloc[[index]]
                    print'yy'
                    #textOutput.write('%s\n' % newReader.Time_stamp.iloc[[x+1]])
                    textOutput.write('^\n')
                    firstKP = False
                    secondKP = False

            # If a second keypress was found, and becomes missing due to a CLEAR, reset boolean
            elif (secondKP == True and pd.isnull(newReader['Second'].iloc[index]) == True):
                secondKP = False
                clr = True

            ## Third Asterisk Occurance
            # Check third column for True
            if (newReader['Third'].iloc[index] == True and pd.isnull(newReader['Fourth'].iloc[index]) == True):

                # Check to see if the third keypress was already found
                if (thirdKP == False):
                    if (clr == True):    
                        clr = False

                    print newReader.Time_stamp.iloc[[index]]
                    print'z'
                    #textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    thirdKP = True

                # If the user presses enter before the four PIN sequence is completed.
                if (newReader['Background'].iloc[x+1] == True and thirdKP == True and secondKP == True and firstKP == True):
                    newReader.Time_stamp.iloc[[x+1]]
                    #textOutput.write('%s\n' % newReader.Time_stamp.iloc[[x+1]])
                    print newReader.Time_stamp.iloc[[index]]
                    print 'zz'
                    textOutput.write('^\n')
                    firstKP = False
                    secondKP = False
                    thirdKP = False

            # If a third keypress was found, and becomes missing due to a CLEAR, reset boolean
            elif (thirdKP == True and pd.isnull(newReader['Fourth'].iloc[index]) == True):
                thirdKP = False
                clr = True

            ## Fourth Asterisk Occurance
            # Check fourth column for True
            # Check to see if fourth keypress was found
            if (newReader['Fourth'].iloc[index] == True and fourthKP == False and pd.isnull(newReader['First'].iloc[index]) == False and
                  pd.isnull(newReader['Second'].iloc[index]) == False and pd.isnull(newReader['Third'].iloc[index]) == False):

                    # If a clear was not taken place, display time at which the fourth asterisk had appeared
                    if (clr == False):
                        print newReader.Time_stamp.iloc[[index]]
                        print 'a'
                        #textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                        fourthKP = True

                    else:
                        clr = False

            # Check to see if a key was removed
            elif (pd.isnull(newReader['Fourth'].iloc[index]) == True and fourthKP == True or newReader['Fourth'].iloc[index] == True 
                  and pd.notnull(newReader['Fourth'].iloc[index]) == True and fourthKP == True):
                
                if (pd.isnull(newReader['First'].iloc[index]) == pd.isnull(newReader['Second'].iloc[index]) 
                    == pd.isnull(newReader['Third'].iloc[index]) == True or newReader['next_and'].iloc[index] == False):
                    # Once all keys were found and the screen clears, reset all booleans
                    firstKP = False
                    secondKP = False
                    thirdKP = False
                    fourthKP = False
                    clr = False
                    print newReader.Time_stamp.iloc[[index]]
                    print 'aa'
                    print '---------------------------------------\n'
                    #textOutput.write('%s\n' % newReader.Time_stamp.iloc[[index]])
                    #textOutput.write('---------------------------------------\n')
                else:
                    fourthKP = False
                    clr = True
                
                # Detect if the fourth asterisk has been cleared
                #elif (pd.notnull(newReader['First'].iloc[index]) == pd.notnull(newReader['Second'].iloc[index]) == pd.notnull(newReader['Third'].iloc[index]) == True):    
            
            #elif (firstKP == True & secondKP == True & thirdKP == True & fourthKP == True and newReader['next_and'].iloc[index] == False):

            #elif (newReader['First'].iloc[index] != newReader['Second'].iloc[index] != newReader['Third'].iloc[index] != newReader['Fourth'].iloc[index]
            #    and firstKP == True and secondKP == True and thirdKP == True and fourthKP == True):
            #    print 'x
            
except:
    print 'End of File'
    
    
with open('../relevantKeyPresses' + videoFileName + '.txt', 'r') as input:
    with open('../cleanedRelevantKeyPresses' + videoFileName + '.txt','wb') as output: 
        for line in input:
            if (line!="Name: Time_stamp, dtype: float64"+"\n"):
                output.write(line)
print 'Complete'